In [ ]:
!pip install transformers datasets

In [1]:
from datasets import load_dataset
from transformers import AutoModel, AutoTokenizer
import itertools
from collections import Counter
import torch
from torch.utils.data import Dataset,DataLoader
import torch.nn.functional as F
import pickle
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
from numpy import linalg as LA
import random
from tqdm import tqdm

from metrics import self_similarity_and_rogue, identifying_rogue_dimensions, mean_pooling, intra_sim_and_norm, anisotropy_baseline

In [2]:
dataset = load_dataset("stsb_multi_mt", name="en", split="test")
sentences = dataset['sentence1'] 
len(sentences)

Found cached dataset stsb_multi_mt (/home/chenghao/.cache/huggingface/datasets/stsb_multi_mt/en/1.0.0/a5d260e4b7aa82d1ab7379523a005a366d9b124c76a5a5cf0c4c5365458b0ba9)


1379

In [3]:
MODEL_NAME = 'sentence-transformers/all-mpnet-base-v2'
#MODEL_NAME = 'sentence-transformers/all-MiniLM-L6-v2'

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModel.from_pretrained(MODEL_NAME,
                                  output_hidden_states = True # returns all hidden-states, enabling analysis on all layers
                                  )
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')# max length should be added here instead of when initializing it

In [5]:
tokenized = encoded_input['input_ids'].tolist()
token_list = list(itertools.chain.from_iterable(tokenized))
counter = Counter(token_list)

print(len(token_list)) #59297
print(counter[2]) # [sep] token = 1379
print(counter[0]) # [cls] token = 1379
print(counter[1]) # padding token = 39705


# get the words to analyze self similarity - appearing in more than 5 contexts (some contexts) at least

index_count = [(index, count) for (index,count) in counter.most_common() if count >=5]
token_list = [index for (index,count) in counter.most_common() if count >=5]
print(len(token_list))

59297
1379
1379
39705
455


In [6]:
def position_list(set_of_token, list_of_tokenized_sentences):

    inference_list = {} ## get a dict to store corresponding position (sentence_index, token_index) of words
    for n in set_of_token:
        position_list = []
        for sen_index, sen in enumerate(list_of_tokenized_sentences):
            if n in sen:
                token_index = sen.index(n)
                position_list.append((sen_index, token_index)) # store the corresponding sentence index and token index of that word
        inference_list[n] = position_list

    return inference_list    

inference_list = position_list(token_list, tokenized)

In [7]:
class MyDataset(Dataset):
    def __init__(self, encoded_inputs):
        self.encoded_inputs = encoded_inputs

    def __len__(self):
        return self.encoded_inputs["input_ids"].shape[0]

    def __getitem__(self, idx):
        return {key: val[idx] for key, val in self.encoded_inputs.items()}

# Initialize dataset and dataloader
dataset = MyDataset(encoded_input)
dataloader = DataLoader(dataset, batch_size=32)  # Adjust batch_size as needed

device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = model.to(device)

# Placeholders for the model outputs
total_output = []
total_hidden_states = []

# Iterate over the dataloader
with torch.no_grad():  # Disable gradient calculation
    for batch in dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}  # Move the batch tensor to the right device
        model_output = model(**batch)
        total_output.append(model_output.last_hidden_state.detach().cpu())
        total_hidden_states.append(tuple(state.detach().cpu() for state in model_output.hidden_states))

# Concatenate all outputs and hidden states
total_output = torch.cat(total_output, dim=0)
total_hidden_states = tuple(torch.cat(state, dim=0) for state in zip(*total_hidden_states))

In [8]:
all_layer_self_sim, self_sim_rogue_dimensions = self_similarity_and_rogue(total_hidden_states, token_list, inference_list, n_layers_start = 0, n_layers = 13, analyze_rogue = True)

Layer Progress: 100%|██████████| 13/13 [01:18<00:00,  6.07s/it]


In [ ]:
all_layer_self_sim[12][0]  ## last layer(12), [cls] token (0), self similarity: 0.049

0.049

In [ ]:
len(self_sim_rogue_dimensions[12][0]) # last layer[12], contribution of each dimension for [cls] token(0)'s self similarity measure

768

In [10]:
all_layer_intra_sim, all_layer_norm = intra_sim_and_norm(total_hidden_states, sentences, encoded_input, n_layers_start = 0, n_layers = 13)

layer progress: 100%|██████████| 13/13 [00:05<00:00,  2.39it/s]


In [23]:
for layer in range(13):
    print('unadjusted intra_sim of layer %s:'%layer, np.mean(all_layer_intra_sim[layer]),'\t', 'average norm of layer %s:'%layer, np.mean(all_layer_norm[layer]))

unadjusted intra_sim of layer 0: 0.2868331 	 average norm of layer 0: 5.5752234
unadjusted intra_sim of layer 1: 0.33597353 	 average norm of layer 1: 6.760648
unadjusted intra_sim of layer 2: 0.3985246 	 average norm of layer 2: 7.3132153
unadjusted intra_sim of layer 3: 0.4290589 	 average norm of layer 3: 7.63464
unadjusted intra_sim of layer 4: 0.44711876 	 average norm of layer 4: 8.217234
unadjusted intra_sim of layer 5: 0.47981295 	 average norm of layer 5: 7.7231436
unadjusted intra_sim of layer 6: 0.50441283 	 average norm of layer 6: 8.357803
unadjusted intra_sim of layer 7: 0.4990974 	 average norm of layer 7: 8.325736
unadjusted intra_sim of layer 8: 0.47839057 	 average norm of layer 8: 8.112216
unadjusted intra_sim of layer 9: 0.49515322 	 average norm of layer 9: 7.813351
unadjusted intra_sim of layer 10: 0.5039894 	 average norm of layer 10: 7.7475214
unadjusted intra_sim of layer 11: 0.53178585 	 average norm of layer 11: 7.194023
unadjusted intra_sim of layer 12: 0.85

In [14]:
np.mean(all_layer_intra_sim[12])

0.8501157

In [8]:
anisotropy_matrix, all_layer_anisotropy = anisotropy_baseline(sentences, total_hidden_states, encoded_input, n_samples = 1000, n_layers = 13)

layer progress: 100%|██████████| 13/13 [00:01<00:00, 10.29it/s]


In [9]:
all_layer_anisotropy

{0: 0.031,
 1: 0.041,
 2: 0.06,
 3: 0.075,
 4: 0.076,
 5: 0.096,
 6: 0.107,
 7: 0.104,
 8: 0.089,
 9: 0.114,
 10: 0.1,
 11: 0.109,
 12: 0.016}

In [21]:
unadjusted_intra_sim = np.array([np.mean(all_layer_intra_sim[layer]) for layer in range(13)])
ani_baseline = np.array([ani for layer, ani in all_layer_anisotropy.items()])
adjusted_intra_sim = unadjusted_intra_sim - ani_baseline

In [24]:
for layer in range(13):
    print('adjusted intra_sim of layer %s:'%layer, adjusted_intra_sim[layer])

adjusted intra_sim of layer 0: 0.25583310747146604
adjusted intra_sim of layer 1: 0.29497353100776674
adjusted intra_sim of layer 2: 0.33852461218833924
adjusted intra_sim of layer 3: 0.3540589094161987
adjusted intra_sim of layer 4: 0.3711187591552734
adjusted intra_sim of layer 5: 0.3838129498958588
adjusted intra_sim of layer 6: 0.39741282987594606
adjusted intra_sim of layer 7: 0.3950974068641663
adjusted intra_sim of layer 8: 0.38939057445526126
adjusted intra_sim of layer 9: 0.38115321850776673
adjusted intra_sim of layer 10: 0.4039893984794617
adjusted intra_sim of layer 11: 0.4227858457565308
adjusted intra_sim of layer 12: 0.8341157164573669
